In [ ]:
import numpy as np

In [7]:
cd

C:\Users\pj


In [8]:
import os
new_directory=(r)
os.chdir(new_directory)

In [9]:
current_directory = os.getcwd()
print("Current Working Directory after changing:", current_directory)

Current Working Directory after changing: C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor


In [13]:
import os

directory_path = r"C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor"
os.chdir(directory_path)

%pwd

'C:\\Users\\pj\\Desktop\\End to End Project\\Laptop_price_predictor'

In [ ]:
            numerical_columns = ['Inches', 'Price']
            categorical_columns = [
                'Company', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory',
        'Gpu', 'OpSys', 'Weight',
            ]

            num_pipeline= Pipeline(
                steps=[
                ("imputer",SimpleImputer(strategy="median")),
                ("scaler",StandardScaler())

                ]
            )

            cat_pipeline=Pipeline(

                steps=[
                ("imputer",SimpleImputer(strategy="most_frequent")),
                ("one_hot_encoder",OneHotEncoder()),
                ("scaler",StandardScaler(with_mean=False))
                ]

            )

    def _clean_and_convert_ram_weight_columns(self, df):
        df[DataTransformation.RAM_COLUMN] = df[DataTransformation.RAM_COLUMN].str.replace('GB', '').astype('int32')
        df[DataTransformation.WEIGHT_COLUMN] = df[DataTransformation.WEIGHT_COLUMN].str.replace('kg', '').astype('float32')
        return df

clean_pipeline=Pipeline(
        
        steps=[
                ("Clean",clean_and_convert_ram_weight_columns)
        ]

)

In [79]:

import sys
from dataclasses import dataclass
import numpy as np 
import pandas as pd
from sklearn.compose import ColumnTransformer
from exception import CustomException
from logger import logging
import os
#from utils import save_object
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler


class DataTransformation(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        logging.info("DataTransformation: Starting data transformation steps...")
        X = X.copy()  # Make a copy of the DataFrame to avoid modifying the original data

        # Apply data transformation steps using separate functions
        X = self.remove_unnamed_column(X)
        X = self.rename_columns(X)        
        X = self.selecting_data(X)
        X = self.remove_units(X)
        X = self.convert_ram_to_int(X)
        X = self.convert_weight_to_float(X)
        X = self.extract_touchscreen_feature(X)
        X = self.extract_ips_feature(X)
        X = self.extract_resolution_features(X)

        return X
    
    def remove_unnamed_column(self, data):
        data.drop(columns=['Unnamed: 0'], inplace=True)
        return data
    logging.info("remove_unnamed_column")

    def rename_columns(self, data):
        # Rename the columns as desired
        data.rename(columns={
            'Inches': 'ScreenSize',
            'ScreenResolution': 'Resolution',
            'Cpu': 'Processor',
            'Ram': 'RAM',
            'Memory': 'Storage',
            'Gpu': 'GraphicsCard',
            'OpSys': 'OperatingSystem',
            'Weight': 'Weight_kg',
        }, inplace=True)
        return data
    logging.info("rename_columns")

    def selecting_data(self, data):
    # No need to select specific columns if you want to keep all
        return data
    logging.info("selecting_data")
    
    def remove_units(self, data):
        data['Ram'] = data['Ram'].str.replace('GB', '')
        data['Weight'] = data['Weight'].str.replace('kg', '')
        return data
    logging.info("remove_units")

    def convert_ram_to_int(self, data):
        data['Ram'] = data['Ram'].astype('int32')
        return data
    logging.info("convert_ram_to_int")

    def convert_weight_to_float(self, data):
        data['Weight'] = data['Weight'].astype('float32')
        return data
    logging.info("convert_weight_to_float")

    def extract_touchscreen_feature(self, data):
        data['TouchScreen'] = data['ScreenResolution'].apply(lambda element: 1 if 'Touchscreen' in element else 0)
        return data
    logging.info("extract_touchscreen_feature")

    def extract_ips_feature(self, data):
        data['IPS'] = data['ScreenResolution'].apply(lambda element: 1 if 'IPS' in element else 0)
        return data
    logging.info("extract_ips_feature")

    def extract_cpu_feature(self, data):
        data['CPU_name'] = data['CPU_name'].apply(lambda text:" ".join(text.split()[:3]))
        return data
    logging.info("extract_ips_feature")

    

    def extract_resolution_features(self, data):
        split_df = data['ScreenResolution'].str.split('x', n=1, expand=True)
        data['X_res'] = split_df[0].str.replace(',', '').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0]).astype('int')
        data['Y_res'] = split_df[1].astype('int')
        data['PPI'] = (((data['X_res'] ** 2 + data['Y_res'] ** 2) ** 0.5) / data['Inches']).astype('float')
        data.drop(columns=['ScreenResolution', 'Inches', 'X_res', 'Y_res'], inplace=True)
        return data
    logging.info("extract_resolution_features")
    

def data_transformation_pipeline(data):
    # Automatically identify numerical and categorical columns
    numerical_columns = data.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
    categorical_columns = data.select_dtypes(include=['object']).columns

    data_transformer = DataTransformation()

    # Create a pipeline to encapsulate the data transformation steps
    data_pipeline = Pipeline(steps=[
        ('data_transform', data_transformer),
        ('scaler', StandardScaler()),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create the ColumnTransformer
    preprocessor = ColumnTransformer(transformers=[
        ('num', data_pipeline, numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])

    logging.info("data_transformation_pipeline: Starting data transformation pipeline...")

    try:
        processed_data = preprocessor.fit_transform(data)
    except KeyError as e:
        logging.error(f"data_transformation_pipeline: KeyError - {e}")
        raise  # Re-raise the exception to see the full traceback

    logging.info("data_transformation_pipeline: Data transformation pipeline completed.")

    return processed_data

except Exception as e:
raise CustomException(e,sys)

In [96]:
data = pd.read_csv(r"C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor\Notebook\data\laptop_data.csv")
data.head()

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,256GB SSD,Intel HD Graphics 620,No OS,1.86,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,512GB SSD,AMD Radeon Pro 455,macOS,1.83,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,96095.8080


In [97]:
class YourClass:
    def remove_units(self, data):
        data['Ram'] = data['Ram'].str.replace('GB', '')
        data['Weight'] = data['Weight'].str.replace('kg', '')
        return data

# Assuming 'data' is your DataFrame
your_object = YourClass()
df = your_object.remove_units(data)
df.info()
df.head()


In [24]:
import numpy as np 
import pandas as pd
data = pd.read_csv(r"C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor\Notebook\data\laptop_data.csv")
data.head()

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080


In [25]:
class YourClass:
    def remove_unnamed_column(self, data):
        data.drop(columns=['Unnamed: 0'], inplace=True)
        return data

    def rename_columns(self, data):
        # Rename the columns as desired
        data.rename(columns={
            'Inches': 'ScreenSize',
            'ScreenResolution': 'Resolution',
            'Cpu': 'Processor',
            'Ram': 'RAM',
            'Memory': 'Storage',
            'Gpu': 'GraphicsCard',
            'OpSys': 'OperatingSystem',
            'Weight': 'Weight_kg',
        }, inplace=True)
        return data

    def remove_units(self, data):
        data['RAM'] = data['RAM'].str.replace('GB', '')
        data['Weight_kg'] = data['Weight_kg'].str.replace('kg', '')
        return data

    def convert_ram_to_int(self, data):
        data['RAM'] = data['RAM'].astype('int32')
        return data

    def convert_weight_to_float(self, data):
        data['Weight_kg'] = data['Weight_kg'].astype('float32')
        return data

    def extract_touchscreen_feature(self, data):
        data['TouchScreen'] = data['Resolution'].apply(lambda element: 1 if 'Touchscreen' in element else 0)
        return data

    def extract_ips_feature(self, data):
        data['IPS'] = data['Resolution'].apply(lambda element: 1 if 'IPS' in element else 0)
        return data

    def extract_cpu_feature(self, data):
        data['Processor'] = data['Processor'].apply(lambda text: " ".join(text.split()[:3]))
        return data

    def extract_resolution_features(self, data):
        split_df = data['Resolution'].str.split('x', n=1, expand=True)
        data['X_res'] = split_df[0].str.replace(',', '').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0]).astype('int')
        data['Y_res'] = split_df[1].astype('int')
        data['PPI'] = (((data['X_res'] ** 2 + data['Y_res'] ** 2) ** 0.5) / data['ScreenSize']).astype('float')
        data.drop(columns=['Resolution', 'ScreenSize', 'X_res', 'Y_res'], inplace=True)
        return data
your_object = YourClass()



In [26]:
# Step 1: Remove the 'Unnamed: 0' column
df = your_object.remove_unnamed_column(data)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1303 non-null   object 
 1   TypeName          1303 non-null   object 
 2   Inches            1303 non-null   float64
 3   ScreenResolution  1303 non-null   object 
 4   Cpu               1303 non-null   object 
 5   Ram               1303 non-null   object 
 6   Memory            1303 non-null   object 
 7   Gpu               1303 non-null   object 
 8   OpSys             1303 non-null   object 
 9   Weight            1303 non-null   object 
 10  Price             1303 non-null   float64
dtypes: float64(2), object(9)
memory usage: 112.1+ KB
None
  Company   TypeName  Inches                    ScreenResolution  \
0   Apple  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook    13.3                            1440x900   
2      HP   Noteboo

In [27]:
# Step 2: Rename the columns as desired
df = your_object.rename_columns(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   object 
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   object 
 10  Price            1303 non-null   float64
dtypes: float64(2), object(9)
memory usage: 112.1+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook        13.3                            1440x900   
2      HP   Notebook

In [28]:
# Step 4: Remove units from 'RAM' and 'Weight_kg' columns
df = your_object.remove_units(df)
print(df.info())
print(df.head())
print('(------)')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   object 
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   object 
 10  Price            1303 non-null   float64
dtypes: float64(2), object(9)
memory usage: 112.1+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook        13.3                            1440x900   
2      HP   Notebook

In [29]:
# Step 5: Convert 'RAM' column to integer type
df = your_object.convert_ram_to_int(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   int32  
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   object 
 10  Price            1303 non-null   float64
dtypes: float64(2), int32(1), object(8)
memory usage: 107.0+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook        13.3                            1440x900   
2      HP 

In [30]:
# Step 6: Convert 'Weight_kg' column to float type
df = your_object.convert_weight_to_float(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   int32  
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   float32
 10  Price            1303 non-null   float64
dtypes: float32(1), float64(2), int32(1), object(7)
memory usage: 101.9+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ultrabook        13.3                            1440x900  

In [31]:
# Step 7: Extract the 'TouchScreen' feature
df = your_object.extract_touchscreen_feature(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   int32  
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   float32
 10  Price            1303 non-null   float64
 11  TouchScreen      1303 non-null   int64  
dtypes: float32(1), float64(2), int32(1), int64(1), object(7)
memory usage: 112.1+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Ult

In [32]:
# Step 8: Extract the 'IPS' feature
df = your_object.extract_ips_feature(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   int32  
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   float32
 10  Price            1303 non-null   float64
 11  TouchScreen      1303 non-null   int64  
 12  IPS              1303 non-null   int64  
dtypes: float32(1), float64(2), int32(1), int64(2), object(7)
memory usage: 122.3+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Pa

In [33]:
# Step 9: Extract the first three words of 'Processor' as 'Processor'
df = your_object.extract_cpu_feature(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   ScreenSize       1303 non-null   float64
 3   Resolution       1303 non-null   object 
 4   Processor        1303 non-null   object 
 5   RAM              1303 non-null   int32  
 6   Storage          1303 non-null   object 
 7   GraphicsCard     1303 non-null   object 
 8   OperatingSystem  1303 non-null   object 
 9   Weight_kg        1303 non-null   float32
 10  Price            1303 non-null   float64
 11  TouchScreen      1303 non-null   int64  
 12  IPS              1303 non-null   int64  
dtypes: float32(1), float64(2), int32(1), int64(2), object(7)
memory usage: 122.3+ KB
None
  Company   TypeName  ScreenSize                          Resolution  \
0   Apple  Ultrabook        13.3  IPS Pa

In [34]:
# Step 10: Extract resolution features and drop unnecessary columns
df = your_object.extract_resolution_features(df)
print(df.info())
print(df.head())
print('(------)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Company          1303 non-null   object 
 1   TypeName         1303 non-null   object 
 2   Processor        1303 non-null   object 
 3   RAM              1303 non-null   int32  
 4   Storage          1303 non-null   object 
 5   GraphicsCard     1303 non-null   object 
 6   OperatingSystem  1303 non-null   object 
 7   Weight_kg        1303 non-null   float32
 8   Price            1303 non-null   float64
 9   TouchScreen      1303 non-null   int64  
 10  IPS              1303 non-null   int64  
 11  PPI              1303 non-null   float64
dtypes: float32(1), float64(2), int32(1), int64(2), object(6)
memory usage: 112.1+ KB
None
  Company   TypeName      Processor  RAM              Storage  \
0   Apple  Ultrabook  Intel Core i5    8            128GB SSD   
1   Apple  Ultrabook  Intel 

In [ ]:
import sys
from dataclasses import dataclass
import numpy as np 
import pandas as pd
from sklearn.compose import ColumnTransformer
from exception import CustomException
from logger import logging
import os
#from utils import save_object
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler


class DataTransformation(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        logging.info("DataTransformation: Starting data transformation steps...")
        X = X.copy()  # Make a copy of the DataFrame to avoid modifying the original data

        # Apply data transformation steps using separate functions
        X = self.remove_unnamed_column(X)
        X = self.rename_columns(X)        
        X = self.selecting_data(X)
        X = self.remove_units(X)
        X = self.convert_ram_to_int(X)
        X = self.convert_weight_to_float(X)
        X = self.extract_touchscreen_feature(X)
        X = self.extract_ips_feature(X)
        X = self.extract_resolution_features(X)

        return X
    
    def remove_unnamed_column(self, data):
        data.drop(columns=['Unnamed: 0'], inplace=True)
        return data
    logging.info("remove_unnamed_column")

    def rename_columns(self, data):
        # Rename the columns as desired
        data.rename(columns={
            'Inches': 'ScreenSize',
            'ScreenResolution': 'Resolution',
            'Cpu': 'Processor',
            'Ram': 'RAM',
            'Memory': 'Storage',
            'Gpu': 'GraphicsCard',
            'OpSys': 'OperatingSystem',
            'Weight': 'Weight_kg',
        }, inplace=True)
        return data
    logging.info("rename_columns")

    def selecting_data(self, data):
    # No need to select specific columns if you want to keep all
        return data
    logging.info("selecting_data")
    
    def remove_units(self, data):
        data['Ram'] = data['Ram'].str.replace('GB', '')
        data['Weight'] = data['Weight'].str.replace('kg', '')
        return data
    logging.info("remove_units")

    def convert_ram_to_int(self, data):
        data['Ram'] = data['Ram'].astype('int32')
        return data
    logging.info("convert_ram_to_int")

    def convert_weight_to_float(self, data):
        data['Weight'] = data['Weight'].astype('float32')
        return data
    logging.info("convert_weight_to_float")

    def extract_touchscreen_feature(self, data):
        data['TouchScreen'] = data['ScreenResolution'].apply(lambda element: 1 if 'Touchscreen' in element else 0)
        return data
    logging.info("extract_touchscreen_feature")

    def extract_ips_feature(self, data):
        data['IPS'] = data['ScreenResolution'].apply(lambda element: 1 if 'IPS' in element else 0)
        return data
    logging.info("extract_ips_feature")

    def extract_cpu_feature(self, data):
        data['CPU_name'] = data['CPU_name'].apply(lambda text:" ".join(text.split()[:3]))
        return data
    logging.info("extract_ips_feature")

    

    def extract_resolution_features(self, data):
        split_df = data['ScreenResolution'].str.split('x', n=1, expand=True)
        data['X_res'] = split_df[0].str.replace(',', '').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0]).astype('int')
        data['Y_res'] = split_df[1].astype('int')
        data['PPI'] = (((data['X_res'] ** 2 + data['Y_res'] ** 2) ** 0.5) / data['Inches']).astype('float')
        data.drop(columns=['ScreenResolution', 'Inches', 'X_res', 'Y_res'], inplace=True)
        return data
    logging.info("extract_resolution_features")
    

def data_transformation_pipeline(data):
    # Automatically identify numerical and categorical columns
    numerical_columns = data.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
    categorical_columns = data.select_dtypes(include=['object']).columns

    data_transformer = DataTransformation()

    # Create a pipeline to encapsulate the data transformation steps
    data_pipeline = Pipeline(steps=[
        ('data_transform', data_transformer),
        ('scaler', StandardScaler()),
        ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create the ColumnTransformer
    preprocessor = ColumnTransformer(transformers=[
        ('num', data_pipeline, numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ])

    logging.info("data_transformation_pipeline: Starting data transformation pipeline...")

    try:
        processed_data = preprocessor.fit_transform(data)
    except KeyError as e:
        logging.error(f"data_transformation_pipeline: KeyError - {e}")
        raise  # Re-raise the exception to see the full traceback

    logging.info("data_transformation_pipeline: Data transformation pipeline completed.")

    return processed_data

   except Exception as e:
      raise CustomException(e,sys)


def initiate_data_transformation(self,train_path,test_path):
    try:
            train_df=pd.read_csv(train_path)
            test_df=pd.read_csv(test_path)

            logging.info("Read train and test data completed")

            logging.info("Obtaining preprocessing object")

            preprocessing_obj=self.get_data_transformer_object()

            target_column_name="math_score"
            numerical_columns = ["writing_score", "reading_score"]

            input_feature_train_df=train_df.drop(columns=[target_column_name],axis=1)
            target_feature_train_df=train_df[target_column_name]

            input_feature_test_df=test_df.drop(columns=[target_column_name],axis=1)
            target_feature_test_df=test_df[target_column_name]

            logging.info(
                f"Applying preprocessing object on training dataframe and testing dataframe."
            )

            input_feature_train_arr=preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr=preprocessing_obj.transform(input_feature_test_df)

            train_arr = np.c_[
                input_feature_train_arr, np.array(target_feature_train_df)
            ]
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

            logging.info(f"Saved preprocessing object.")

            save_object(

                file_path=self.data_transformation_config.preprocessor_obj_file_path,
                obj=preprocessing_obj

            )

            return (
                train_arr,
                test_arr,
                self.data_transformation_config.preprocessor_obj_file_path,
            )
        except Exception as e:
            raise CustomException(e,sys)

In [ ]:
import sys
import os
from dataclasses import dataclass
import numpy as np 
import pandas as pd

from sklearn.compose import ColumnTransformer
from exception import CustomException
from logger import logging
from utils.utils import save_object
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

@dataclass
class DataTransformationConfig:
    preprocessor_obj_file_path: str = os.path.join('artifacts', 'preprocessor.pkl')

class DataTransformation(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        logging.info("DataTransformation: Starting data transformation steps...")
        X = X.copy()  # Make a copy of the DataFrame to avoid modifying the original data
        required_columns = ['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']

        # Check if all the required columns are present in the DataFrame
        missing_columns = set(required_columns) - set(X.columns)
        if missing_columns:
            raise ValueError(f"Missing columns in the DataFrame: {missing_columns}")

        # Selecting and reordering columns directly
        X = X[required_columns]

        # Apply other transformations only if certain columns are present
        if 'Unnamed: 0' in X.columns:
            X.drop(columns=['Unnamed: 0'], inplace=True)
            logging.info("remove_unnamed_column")

        if 'ScreenResolution' in X.columns:
            if 'ScreenResolution' in X.columns:
                X.rename(columns={'ScreenResolution': 'Resolution'}, inplace=True)
            X.rename(columns={
                'Company': 'Brand',
                'TypeName': 'Type',
                'Inches': 'ScreenSize',
                'Ram': 'RAM',
                'Memory': 'Storage',
                'Gpu': 'GPU',
                'OpSys': 'OperatingSystem',
                'Weight': 'Weight',
                'Price_euros': 'Price'
            }, inplace=True)
            logging.info("rename_columns")

        if 'RAM' in X.columns:
            X['RAM'] = X['RAM'].str.replace('GB', '')
            X['RAM'] = X['RAM'].astype('int32')
            logging.info("remove_units and convert_ram_to_int")

        if 'Weight_kg' in X.columns:
            X['Weight_kg'] = X['Weight_kg'].str.replace('kg', '')
            X['Weight_kg'] = X['Weight_kg'].astype('float32')
            logging.info("convert_weight_to_float")

        if 'Resolution' in X.columns:
            X['TouchScreen'] = X['Resolution'].apply(lambda element: 1 if 'Touchscreen' in element else 0)
            X['IPS'] = X['Resolution'].apply(lambda element: 1 if 'IPS' in element else 0)
            logging.info("extract_touchscreen_feature and extract_ips_feature")

        if 'CPU' in X.columns:
            print("Columns before extracting CPU feature:", X.columns)
            X['Processor'] = X['CPU'].apply(lambda text: " ".join(text.split()[:3]))
            print("Columns after extracting CPU feature:", X.columns)
            logging.info("extract_cpu_feature")

        if 'Resolution' in X.columns:
            split_df = X['Resolution'].str.split('x', n=1, expand=True)
            X['X_res'] = split_df[0].str.replace(',', '').str.findall(r'(\d+\.?\d+)').apply(lambda x: x[0]).astype('int')
            X['Y_res'] = split_df[1].astype('int')
            X['PPI'] = (((X['X_res'] ** 2 + X['Y_res'] ** 2) ** 0.5) / X['ScreenSize']).astype('float')
            # Drop only the columns that are no longer needed for further processing
            X.drop(columns=['X_res', 'Y_res'], inplace=True)
            logging.info("extract_resolution_features")

        if 'CPU_name' in X.columns:
            listtoapply = ['HDD', 'SSD', 'Hybrid', 'FlashStorage']
            for value in listtoapply:
                X['Layer1'+value] = X['first'].apply(lambda x: 1 if value in x else 0)

            X['first'] = X['first'].str.replace(r'\D','')
            X['first'] = X['first'].astype('int')

            listtoapply1 = ['HDD', 'SSD', 'Hybrid', 'FlashStorage']
            X['Second'] = X['Second'].fillna("0")
            for value in listtoapply1:
                X['Layer2'+value] = X['Second'].apply(lambda x: 1 if value in x else 0)

            X['Second'] = X['Second'].str.replace(r'\D','')
            X['Second'] = X['Second'].astype('int')

            # Multiplying the elements and storing the result in subsequent columns
            X["HDD"] = (X["first"] * X["Layer1HDD"] + X["Second"] * X["Layer2HDD"])
            X["SSD"] = (X["first"] * X["Layer1SSD"] + X["Second"] * X["Layer2SSD"])

            # ... (add similar lines for Hybrid and Flash_Storage if needed)

            # Dropping unnecessary columns
            X.drop(columns=['first', 'Second', 'Layer1HDD', 'Layer1SSD', 'Layer1Hybrid',
                            'Layer1FlashStorage', 'Layer2HDD', 'Layer2SSD', 'Layer2Hybrid',
                            'Layer2FlashStorage'], inplace=True)
            X.drop(columns=['Memory'], inplace=True)
            X.drop(columns=['Hybrid', 'Flash_Storage'], inplace=True)

            # ... (add any additional processing steps if needed)

            # Processing GPU column
            X['Gpu brand'] = X['Gpu'].apply(lambda x: x.split()[0])
            X.drop(columns=['Gpu'], inplace=True)

            # Setting OpSys category
            X['OpSys'] = X['OpSys'].apply(lambda x: self.setcategory(x))

        return X

    def data_transformation_pipeline(self, data):
        # Automatically identify numerical and categorical columns
        numerical_columns = data.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
        categorical_columns = data.select_dtypes(include=['object']).columns

        data_transformer = DataTransformation()

        # Create a pipeline to encapsulate the data transformation steps
        data_pipeline = Pipeline(steps=[
            ('data_transform', data_transformer),
            ('scaler', StandardScaler()),
            ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
        ])

        # Create the ColumnTransformer
        preprocessor = ColumnTransformer(transformers=[
            ('num', data_pipeline, numerical_columns),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
        ])

        logging.info("data_transformation_pipeline: Starting data transformation pipeline...")

        try:
            processed_data = preprocessor.fit_transform(data)
        except KeyError as e:
            logging.error(f"data_transformation_pipeline: KeyError - {e}")
            raise  # Re-raise the exception to see the full traceback

        logging.info("data_transformation_pipeline: Data transformation pipeline completed.")

        return processed_data

    def initiate_data_transformation(self, train_path, test_path):
        try:
            train_df = pd.read_csv(train_path)
            test_df = pd.read_csv(test_path)

            logging.info("Read train and test data completed")

            logging.info("Obtaining preprocessing object")

            preprocessing_obj = self.data_transformation_pipeline(train_df)

            target_column_name = "Price"
            numerical_columns = numerical_columns

            input_feature_train_df = train_df.drop(columns=[target_column_name], axis=1)
            target_feature_train_df = train_df[target_column_name]

            input_feature_test_df = test_df.drop(columns=[target_column_name], axis=1)
            target_feature_test_df = test_df[target_column_name]

            logging.info(
                f"Applying preprocessing object on training dataframe and testing dataframe."
            )

            input_feature_train_arr = preprocessing_obj.fit_transform(input_feature_train_df)
            input_feature_test_arr = preprocessing_obj.transform(input_feature_test_df)

            train_arr = np.c_[input_feature_train_arr, np.array(target_feature_train_df)]
            test_arr = np.c_[input_feature_test_arr, np.array(target_feature_test_df)]

            logging.info(f"Saved preprocessing object.")

            save_object(
                file_path=self.data_transformation_config.preprocessor_obj_file_path,
                obj=preprocessing_obj
            )

            return (
                train_arr,
                test_arr,
                self.data_transformation_config.preprocessor_obj_file_path,
            )
        except Exception as e:
            raise CustomException(e, sys)


In [40]:
import pandas as pd
df = pd.read_csv(r"C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor\artifacts\test.csv")
df.head(2)

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,479,Toshiba,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.05kg,89084.16
1,1022,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg,61218.72


In [41]:
import pandas as pd
df = pd.read_csv(r"C:\Users\pj\Desktop\End to End Project\Laptop_price_predictor\artifacts\test.csv")
df.head(2)

,Unnamed: 0,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,479,Toshiba,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.05kg,89084.16
1,1022,HP,Notebook,13.3,IPS Panel Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg,61218.72
